<a href="https://colab.research.google.com/github/deenukhan/deep_learning/blob/main/4_1_dog_vs_cat_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Notebook is inspired by 
# https://keras.io/guides/transfer_learning/
# https://www.coursera.org/learn/convolutional-neural-networks-tensorflow/home/welcome

# In this Notebook we're going to try few State of The Computer Vision Model, Like Inception V3, Xception, VGG and more.
# We will also be trying several Transfer learning techniques, like, using Transfer Learning for prediction, feature extraction 
# and  Fineturning the Transfer learning model according to our data
# So Let's Get Started

## **Transfer Learning** 

The typical transfer-learning workflow
This leads us to how a typical transfer learning workflow can be implemented in Keras:

1. Instantiate a base model and load pre-trained weights into it.
2. Freeze all layers in the base model by setting trainable = False.
3. Create a new model on top of the output of one (or several) layers from the base model.
4. Train your new model on your new dataset.

Note that an alternative, more lightweight workflow could also be:

1. Instantiate a base model and load pre-trained weights into it.
2. Run your new dataset through it and record the output of one (or several) layers from the base model. This is called feature extraction.
3. Use that output as input data for a new, smaller model.

A key advantage of that second workflow is that you only run the base model once on your data, rather than once per epoch of training. So it's a lot faster & cheaper.

In [13]:
# Import Some Libraries
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten

In [14]:
# We will be using dog_vs_cat dataset for this Notebook
# So Let's download the dataset and extract it
dog_vs_cat_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
keras.utils.get_file('dog_vs_cat.zip', dog_vs_cat_url, cache_dir = '/content/')

zipfile.ZipFile('/content/datasets/dog_vs_cat.zip', 'r').extractall('/content/datasets/')

'/content/datasets/dog_vs_cat.zip'

In [16]:
# # Let's Check the Filenames 
# main_dir = '/content/datasets/cats_and_dogs_filtered'

training_cats  = os.listdir('/content/datasets/cats_and_dogs_filtered/train/cats')
training_dogs  = os.listdir('/content/datasets/cats_and_dogs_filtered/train/dogs')
val_cats  = os.listdir('/content/datasets/cats_and_dogs_filtered/validation/cats')
val_dogs  = os.listdir('/content/datasets/cats_and_dogs_filtered/validation/dogs')

In [17]:
# Printing some length and few filenames 
print("Training Cats Filenames : ", training_cats[:5])
print("Training Dogs Filenames : ", training_dogs[:5])
print("Validation Cats Filenames: ", val_cats[:5])
print("Training Dogs Filenamse: ", val_dogs[:5])

print("Total Training Dogs : ", len(training_dogs))
print("Total Training Cats : ", len(training_cats))
print("Total Validation Dogs : ", len(val_cats))
print("Total Validation Cats : ", len(val_dogs))

Training Cats Filenames :  ['cat.657.jpg', 'cat.639.jpg', 'cat.776.jpg', 'cat.411.jpg', 'cat.298.jpg']
Training Dogs Filenames :  ['dog.40.jpg', 'dog.684.jpg', 'dog.628.jpg', 'dog.320.jpg', 'dog.618.jpg']
Validation Cats Filenames:  ['cat.2498.jpg', 'cat.2117.jpg', 'cat.2465.jpg', 'cat.2122.jpg', 'cat.2040.jpg']
Training Dogs Filenamse:  ['dog.2467.jpg', 'dog.2190.jpg', 'dog.2453.jpg', 'dog.2129.jpg', 'dog.2002.jpg']
Total Training Dogs :  1000
Total Training Cats :  1000
Total Validation Dogs :  500
Total Validation Cats :  500


In [18]:
# Now Let's create the Pipeline for our Model, we are using ImageDataGenerator for our Model
# We are also using some Data Augmentation like flip, shear, rotation etc.
img_train_gen = keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255,
                horizontal_flip = True,
                width_shift_range = 0.2,
                vertical_flip = True,
                shear_range = 0.2,
                rotation_range = 40,
                zoom_range = 0.2 
)
img_val_gen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

img_train_gen = img_train_gen.flow_from_directory(
                "/content/datasets/cats_and_dogs_filtered/train",
                target_size = (150, 150),
                class_mode = 'binary', batch_size = 32,
                shuffle = True
                )
img_val_gen = img_val_gen.flow_from_directory(
                "/content/datasets/cats_and_dogs_filtered/validation",
                target_size = (150, 150),
                class_mode = 'binary', batch_size = 32,
                shuffle = True
                )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## **Inception V3 Model**

In [19]:
# You can read more about this model on this link https://keras.io/api/applications/inceptionv3/ 
# First We need to Load the Model 

base_model = keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

# for layers in base_model.layers:
#     layers.trainable = False

In [22]:
# Here We are using only one Dense Layer after the InceptionV3 Model to Just to keep thing Simple
inputs = Input(shape=(150,150,3))
x = base_model(inputs)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

In [12]:
# Let's Compile and Train the Model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(img_train_gen, epochs = 10, validation_data=img_val_gen)

# Here We can see that Our Model has got 95 Percent Accuracy In Just 10 epochs
# That is Amazing, We got around 70 Percent accuracy when we built our model earlier from scratch in 10 epochs

Epoch 1/10
63/63 [==============================] - 61s 410ms/step - loss: 3.9260 - accuracy: 0.6583 - val_loss: 0.1692 - val_accuracy: 0.9390
Epoch 2/10
63/63 [==============================] - 22s 356ms/step - loss: 0.3784 - accuracy: 0.8591 - val_loss: 0.1638 - val_accuracy: 0.9370
Epoch 3/10
63/63 [==============================] - 23s 363ms/step - loss: 0.3282 - accuracy: 0.8630 - val_loss: 0.1177 - val_accuracy: 0.9560
Epoch 4/10
63/63 [==============================] - 22s 353ms/step - loss: 0.2507 - accuracy: 0.8974 - val_loss: 0.1764 - val_accuracy: 0.9300
Epoch 5/10
63/63 [==============================] - 23s 358ms/step - loss: 0.2928 - accuracy: 0.8851 - val_loss: 0.1645 - val_accuracy: 0.9370
Epoch 6/10
63/63 [==============================] - 22s 351ms/step - loss: 0.2535 - accuracy: 0.8878 - val_loss: 0.1131 - val_accuracy: 0.9570
Epoch 7/10
63/63 [==============================] - 22s 356ms/step - loss: 0.2438 - accuracy: 0.9084 - val_loss: 0.1232 - val_accuracy: 0.9540

## **Xception Model**

In [25]:
# And with Just a Single Word Change We can implement Another Model That is Amazing
# You can read more about this model on this link https://keras.io/api/applications/xception/ 
# First We need to Load the Model 

base_model = keras.applications.Xception(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

# for layers in base_model.layers:
#     layers.trainable = False

In [26]:
# Here We are using only one Dense Layer after the Xception Model to Just to keep thing Simple
keras.backend.clear_session()

inputs = Input(shape=(150,150,3))
x = base_model(inputs)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)


# Let's Compile and Train the Model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(img_train_gen, epochs = 10, validation_data=img_val_gen)

# It Feels Like We don't even need to Train our Dense layer more than 1 epoch, because we are getting around 95% accuracy Just like thatn
# And This model and Given us more than 96% Accuracy on Validation Dataset, That is just great!!

Epoch 1/10
63/63 [==============================] - 30s 408ms/step - loss: 2.5784 - accuracy: 0.7288 - val_loss: 0.1757 - val_accuracy: 0.9410
Epoch 2/10
63/63 [==============================] - 24s 375ms/step - loss: 0.4018 - accuracy: 0.8581 - val_loss: 0.1523 - val_accuracy: 0.9500
Epoch 3/10
63/63 [==============================] - 24s 374ms/step - loss: 0.2895 - accuracy: 0.8830 - val_loss: 0.1648 - val_accuracy: 0.9430
Epoch 4/10
63/63 [==============================] - 24s 373ms/step - loss: 0.2515 - accuracy: 0.8935 - val_loss: 0.1257 - val_accuracy: 0.9550
Epoch 5/10
63/63 [==============================] - 24s 373ms/step - loss: 0.2519 - accuracy: 0.9008 - val_loss: 0.1560 - val_accuracy: 0.9450
Epoch 6/10
63/63 [==============================] - 24s 373ms/step - loss: 0.2596 - accuracy: 0.8993 - val_loss: 0.1032 - val_accuracy: 0.9630
Epoch 7/10
63/63 [==============================] - 24s 375ms/step - loss: 0.2426 - accuracy: 0.8892 - val_loss: 0.1247 - val_accuracy: 0.9480

## **VGG16 Model**

In [29]:
# And with Just a Single Word Change We can implement Another Model That is Amazing
# You can read more about this model on this link https://keras.io/api/applications/vgg/#vgg16-function 
# First We need to Load the Model 

base_model = keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

# for layers in base_model.layers:
#     layers.trainable = False

In [30]:
# Here We are using only one Dense Layer after the VGG16 Model to Just to keep thing Simple
keras.backend.clear_session()

inputs = Input(shape=(150,150,3))
x = base_model(inputs, training= False)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)


# Let's Compile and Train the Model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(img_train_gen, epochs = 10, validation_data=img_val_gen)


Epoch 1/10
63/63 [==============================] - 24s 372ms/step - loss: 0.7899 - accuracy: 0.6451 - val_loss: 0.3440 - val_accuracy: 0.8460
Epoch 2/10
63/63 [==============================] - 23s 368ms/step - loss: 0.4654 - accuracy: 0.7743 - val_loss: 0.4858 - val_accuracy: 0.7630
Epoch 3/10
63/63 [==============================] - 23s 369ms/step - loss: 0.4272 - accuracy: 0.7901 - val_loss: 0.3104 - val_accuracy: 0.8650
Epoch 4/10
63/63 [==============================] - 23s 369ms/step - loss: 0.3784 - accuracy: 0.8349 - val_loss: 0.3184 - val_accuracy: 0.8610
Epoch 5/10
63/63 [==============================] - 23s 370ms/step - loss: 0.3651 - accuracy: 0.8251 - val_loss: 0.3254 - val_accuracy: 0.8630
Epoch 6/10
63/63 [==============================] - 23s 371ms/step - loss: 0.3799 - accuracy: 0.8303 - val_loss: 0.3003 - val_accuracy: 0.8620
Epoch 7/10
63/63 [==============================] - 23s 370ms/step - loss: 0.3483 - accuracy: 0.8375 - val_loss: 0.3076 - val_accuracy: 0.8670

## **Fine Tune VGG-16 Model**

Once your model has converged on the new data, you can try to unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate.

This is an optional last step that can potentially give you incremental improvements. It could also potentially lead to quick overfitting -- keep that in mind.

Finally, let's unfreeze the base model and train the entire model end-to-end with a low learning rate.

Importantly, although the base model becomes trainable, it is still running in inference mode since we passed training=False when calling it when we built the model. This means that the batch normalization layers inside won't update their batch statistics. If they did, they would wreck havoc on the representations learned by the model so far.

In [ ]:
base_model.trainable = True

In [31]:
# Let's Compile and Train the Model
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(img_train_gen, epochs = 10, validation_data=img_val_gen)

# Here We can see that accuracy has increased a little bit, but we can train it for more epoch and change the paramaters for better results

Epoch 1/10
63/63 [==============================] - 24s 374ms/step - loss: 0.3300 - accuracy: 0.8593 - val_loss: 0.3100 - val_accuracy: 0.8730
Epoch 2/10
63/63 [==============================] - 23s 365ms/step - loss: 0.3366 - accuracy: 0.8535 - val_loss: 0.3119 - val_accuracy: 0.8740
Epoch 3/10
63/63 [==============================] - 23s 364ms/step - loss: 0.3274 - accuracy: 0.8538 - val_loss: 0.3102 - val_accuracy: 0.8760
Epoch 4/10
63/63 [==============================] - 23s 362ms/step - loss: 0.3233 - accuracy: 0.8582 - val_loss: 0.3091 - val_accuracy: 0.8750
Epoch 5/10
63/63 [==============================] - 23s 365ms/step - loss: 0.3230 - accuracy: 0.8521 - val_loss: 0.3072 - val_accuracy: 0.8760
Epoch 6/10
63/63 [==============================] - 23s 363ms/step - loss: 0.3219 - accuracy: 0.8686 - val_loss: 0.3096 - val_accuracy: 0.8760
Epoch 7/10
63/63 [==============================] - 23s 362ms/step - loss: 0.3342 - accuracy: 0.8560 - val_loss: 0.3078 - val_accuracy: 0.8780